---
# Section 1.6: Sparse Positive Definite Systems
---

If an $n \times n$ symmetric positive definite matrix $A$ has the **arrowhead** structure ($* =$ nonzero entry),

$$
A = 
\begin{bmatrix}
* &   &   &   &  *\\
  & * &   &   &  *\\
  &   & * &   &  *\\
  &   &   & * &  *\\
* & * & * & * &  *\\
\end{bmatrix}
$$

then its Cholesky factor $R$ will have **zero fill-in**:

$$
R = 
\begin{bmatrix}
* &   &   &   &  *\\
  & * &   &   &  *\\
  &   & * &   &  *\\
  &   &   & * &  *\\
  &   &   &   &  *\\
\end{bmatrix}.
$$

However, if we swap the first and last rows and columns we get

$$
A = 
\begin{bmatrix}
* & * & * & * &  *\\
* & * &   &   &   \\
* &   & * &   &   \\
* &   &   & * &   \\
* &   &   &   &  *\\
\end{bmatrix}
$$

and its Cholesky factor $R$ will be **completely filled in**:

$$
R = 
\begin{bmatrix}
* & * & * & * &  *\\
  & * & * & * &  *\\
  &   & * & * &  *\\
  &   &   & * &  *\\
  &   &   &   &  *\\
\end{bmatrix}.
$$

Thus the order of the equations and variables can make a big difference in the amount of fill-in in the Cholesky factor of $A$.

---

## Sparse Cholesky

In [ ]:
using LinearAlgebra, SparseArrays

In [ ]:
n = 1000
density = 0.001
A = sprandn(n, n, density)
A = A + A' + 10*I

In [ ]:
# Dense Cholesky
Adense = Matrix(A)
F = cholesky(Adense)

R = sparse(F.U)

In [ ]:
R'R ≈ A

In [ ]:
# Sparse Cholesky
F = cholesky(A)

# Permutation of the rows and columns of A
p = F.p

A[p,p]

In [ ]:
Rt = sparse(F.L)
R = sparse(Rt')

In [ ]:
R'R ≈ A[p,p]

---

## Reverse Cuthill-McKee (AMD) reordering

In [ ]:
using SymRCM

In [ ]:
p_symrcm = symrcm(A)

A_symrcm = A[p_symrcm,p_symrcm]

In [ ]:
F = cholesky(Matrix(A_symrcm))

R = sparse(F.U)

---

## Approximate Minimum Degree (AMD) reordering

In [ ]:
using AMD

In [ ]:
p_amd = amd(A)

A_amd = A[p_amd,p_amd]

In [ ]:
F = cholesky(Matrix(A_amd))

R = sparse(F.U)

---

## Symmetric AMD

In [ ]:
p_symamd = symamd(A)

A_symamd = A[p_symamd,p_symamd]

In [ ]:
F = cholesky(Matrix(A_symamd))

R = sparse(F.U)

---

## Benchmark

In [ ]:
using BenchmarkTools

In [ ]:
@btime p_symrcm = symrcm(A)
@btime p_amd = amd(A)
@btime p_symamd = symamd(A);

In [ ]:
Adense = Matrix(A)
A_amd = A[p_amd,p_amd]
A_symamd = A[p_symamd,p_symamd]
A_symrcm = A[p_symrcm,p_symrcm];

In [ ]:
@btime cholesky(Adense)
@btime cholesky(A);

In [ ]:
@btime cholesky(A_symrcm)
@btime cholesky(A_amd)
@btime cholesky(A_symamd);

In [ ]:
@btime cholesky(A, perm=1:n);

In [ ]:
@btime cholesky(A, perm=p_symrcm)
@btime cholesky(A, perm=p_amd)
@btime cholesky(A, perm=p_symamd);

---